In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import pprint

# Import the API keys
from config import Client_ID, Client_Secret

In [2]:
#Getting the access token part
igdb_token_base_url = "https://id.twitch.tv/oauth2/token?"
token_data = {
    "client_id":Client_ID,
    "client_secret":Client_Secret,
	"grant_type":"client_credentials"
}
igdb_token_request = requests.post(igdb_token_base_url, data=token_data)
print(igdb_token_request)

<Response [200]>


In [3]:
#Print the response
igdb_token_response = igdb_token_request.json()
pprint.pprint(igdb_token_response)

{'access_token': 'qa0ulv8hns26zbxnmr7d46qnorsekx',
 'expires_in': 4931125,
 'token_type': 'bearer'}


In [6]:
# Store just the access token in a variable.
access_token = igdb_token_response['access_token']

#The base url for getting information about certain games.
games_base_url = "https://api.igdb.com/v4/games"

# Store all the api-key-like values, i.e., Client_ID and access_token, needed to make an api call to get the actual data in a dictionary.
headers = {
    "Client-ID": Client_ID,
    "Authorization": f"Bearer {access_token}"
}

# The query parameters written in a string.
params = {
    "limit": 500,
    "sort": "total_rating_count desc",
    "where": "(category=0 & status=4 & total_rating!=null & total_rating_count!=null)",
    "fields": "age_ratings.category, age_ratings.rating, game_modes.name, genres.name, involved_companies.company.developed, involved_companies.company.name, involved_companies.company.published, involved_companies.developer, involved_companies.publisher, name, release_dates.human, release_dates.platform.name, release_dates.region, total_rating, total_rating_count"
}

# Placing the parameters in a string, in the required format for the API call to work.
data=""
for key, value in params.items():
    data+=f"{key} {value};\r\n"

games_request = requests.post(games_base_url, headers=headers, data=data)


games_response = {}
if games_request.status_code == 200:
    games_response = games_request.json()
    pprint.pprint(games_response)

[{'age_ratings': [{'category': 1, 'id': 116565, 'rating': 10},
                  {'category': 2, 'id': 116566, 'rating': 3},
                  {'category': 7, 'id': 116567, 'rating': 36},
                  {'category': 5, 'id': 116568, 'rating': 25}],
  'game_modes': [{'id': 1, 'name': 'Single player'},
                 {'id': 2, 'name': 'Multiplayer'},
                 {'id': 3, 'name': 'Co-operative'}],
  'genres': [{'id': 12, 'name': 'Role-playing (RPG)'},
             {'id': 31, 'name': 'Adventure'},
             {'id': 32, 'name': 'Indie'}],
  'id': 104967,
  'involved_companies': [{'company': {'developed': [104967],
                                      'id': 24024,
                                      'name': 'Iron Gate Studios'},
                          'developer': True,
                          'id': 137322,
                          'publisher': False},
                         {'company': {'id': 12186,
                                      'name': 'Coffee Stain Publishi

In [5]:
# Age Ratings category dictionary, which will be used to get the category names from the category IDs stored in games_response.
a_r_cat_dict = {
    "1":"ESRB",
    "2":"PEGI",
    "3":"CERO",
    "4":"USK",
    "5":"GRAC",
    "6":"CLASS_IND",
    "7":"ACB"
}

# Age Ratings dictionary, which will be used to get the rating names from the rating IDs stored in games_response.
a_rat_dict = dict(zip([str(i) for i in range(1,40)],
                    ["Three","Seven","Twelve","Sixteen","Eighteen",
                    "RP","EC","E","E10","T","M","AO",
                    "CERO_A","CERO_B","CERO_C","CERO_D","CERO_Z",
                    "USK_0","USK_6","USK_12","USK_16","USK_18",
                    "GRAC_ALL","GRAC_Twelve","GRAC_Fifteen","GRAC_Eighteen","GRAC_TESTING",
                    "CLASS_IND_L","CLASS_IND_Ten","CLASS_IND_Twelve","CLASS_IND_Fourteen","CLASS_IND_Sixteen","CLASS_IND_Eighteen",
                    "ACB_G","ACB_PG","ACB_M","ACB_MA15","ACB_R18","ACB_RC"]))

# Release date regions dictionary, which will be used to get the region names from the region IDs stored in games_response.
rel_date_reg_dict = {
    "1":"Europe",
    "2":"North America",
    "3":"Australia",
    "4":"New Zealand",
    "5":"Japan",
    "6":"China",
    "7":"Asia",
    "8":"Worldwide",
    "9":"Korea",
    "10":"Brazil"
}

In [ ]:
game_data = []

for game in games_response:
    date = []
    platform = []
    location = []
    name = game["name"] 
    age_ratings = [a_rat_dict[str(rating['rating'] )] for rating in game['age_ratings']]   
    age_rating_category = [a_r_cat_dict[str(rating['category'] )] for rating in game['age_ratings']]
    game_modes = [mode["name"] for mode in game["game_modes"]]
    genres = [genre["name"] for genre in game["genres"]]
    game_ratings = game["total_rating"]
    game_ratings_counts = game["total_rating_count"]
    released_date_info = game["release_dates"]
    
    for date_info in released_date_info:
       date.append(date_info["human"])
       platform.append(date_info["platform"]["name"])
       location.append(rel_date_reg_dict[str(date_info["region"])])

    companies = game["involved_companies"]
    games_developed = []
    games_published = []
    developer_name = []
    publisher_name = []

    for company in companies :
        if company["publisher"] == True:
            publisher_name.append(company["company"]["name"])
            games_published.append(len(company["company"]["published"]))

        if company["developer"] == True:
            developer_name.append(company["company"]["name"])
            games_developed.append(len(company["company"]["developed"]))


   game_data.append({
       "Title" : name,
       "Ratings" : age_ratings,
       "Rating Category" : age_rating_category,
       "Genres" : genres,
       "Platform" : platform,
       "Release Dates" : date,
       "Locations" : location,
       "Publishers" : publisher_name,
       "Games Published" : games_published,
       "Developer": developer_name,
       "Games Developed" : games_developed
   })
       
        


        

   

